https://github.com/graphistry/pygraphistry

https://towardsdatascience.com/visualize-github-social-network-with-pygraphistry-dfc23a38ec8d

In [1]:
%pip install --user graphistry[ai]


     -------------------------------------- 186.5/186.5 kB 2.8 MB/s eta 0:00:00
  Using cached umap-learn-0.5.3.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.3/7.3 MB 6.8 MB/s eta 0:00:00
     -------------------------------------- 155.1/155.1 kB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 86.0/86.0 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.8/2.8 MB 6.6 MB/s eta 0:00:00
  Using cached pynndescent-0.5.7.tar.gz (1.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torch-1.11.0-cp38-cp38-win_amd64.whl (158.0 MB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=754aea6083ab68f6229b83a17fc40

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
causalnlp 0.6.0 requires scipy==1.4.1, but you have scipy 1.8.1 which is incompatible.
causalnex 0.11.0 requires scikit-learn!=0.22.2.post1,!=0.24.1,<0.25.0,>=0.22.0; python_version < "3.9", but you have scikit-learn 1.1.2 which is incompatible.
causalnex 0.11.0 requires scipy<1.7,>=1.2.0, but you have scipy 1.8.1 which is incompatible.
scispacy 0.5.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 3.0.8 which is incompatible.


In [3]:
import graphistry
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="siddharth63", password="sid0112358")

## Hypergraph

In [4]:
%pip install hypernetx -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
import hypernetx as hnx

In [7]:
import pandas as pd

class HyperNetXG:
  
    def __init__(self, graphistry):
        self.graphistry = graphistry      

    def normalize_id(self, id):
        t = type(id)
        if t == float or t == int:
            return '__id__' + str(id)
        return str(id)


    def hypernetx_to_graphistry_bipartite(self, h):

        nodes_df = pd.concat(
            [pd.DataFrame({
                'node': [self.normalize_id(x) for x in list(H.nodes)], 
                'type': 'hypernode'}),
             pd.DataFrame({
                 'node': [self.normalize_id(x) for x in H.edges], 
                 'type': 'hyperedge'})],
            ignore_index=True,
            sort=False)

        edges_df = pd.concat(
            [ pd.DataFrame({'src': [], 'dst': []}) ] + 
            [
                pd.DataFrame({
                    'src': self.normalize_id(k),
                    'dst': [self.normalize_id(x) for x in list(es)]
                })
                for k, es in H.incidence_dict.items()
            ], ignore_index=True, sort=False)

        return self.graphistry.bind(
              source='src', 
              destination='dst', 
              node='node').nodes(nodes_df).edges(edges_df)


    def __hyperedge_to_graph(self, k, es):
        lst = list(es)
        edges_df = pd.concat([
              pd.DataFrame({'src': [], 'dst': [], 'hyperedge': []})] + [
              pd.DataFrame({
                  'src': self.normalize_id(lst[i]), 
                  'dst': [self.normalize_id(x) for x in lst[i+1:]], 
                  'hyperedge': self.normalize_id(k)})
              for i in range(0, len(lst))
          ], ignore_index=True, sort=False)     
        return edges_df

    def hypernetx_to_graphistry_nodes(self, h):
        hg = self.hypernetx_to_graphistry_bipartite(h)
        nodes_df = pd.DataFrame({
              'node': [self.normalize_id(x) for x in list(h.nodes)], 
              'type': 'hypernode'})
        edges_df = pd.concat(
              [pd.DataFrame({'src': [], 'dst': [], 'hyperedge': []})] +
              [
                  self.__hyperedge_to_graph(k, es) 
                  for (k, es) in h.incidence_dict.items() 
              ])
        return self.graphistry.bind(
              source='src', 
              destination='dst', 
              node='node').settings(url_params={'edgeCurvature': 0}).nodes(nodes_df).edges(edges_df)

In [8]:
scenes = [
    ('FN', 'TH'),
    ('TH', 'JV'),
    ('BM', 'FN', 'JA'),
    ('JV', 'JU', 'CH', 'BM'),
    ('JU', 'CH', 'BR', 'CN', 'CC', 'JV', 'BM'),
    ('TH', 'GP'),
    ('GP', 'MP'),
    ('MA', 'GP')
]

H = hnx.Hypergraph(dict(enumerate(scenes)))
hg = HyperNetXG(graphistry)

In [9]:
g = hg.hypernetx_to_graphistry_bipartite(H)
g._nodes.sample(3)

,node,type
11,MP,hypernode
2,JV,hypernode
12,MA,hypernode


In [10]:
g._edges.sample(3)

g.plot()

In [11]:
hg.hypernetx_to_graphistry_bipartite(H.dual())._edges.sample(3)
hg.hypernetx_to_graphistry_bipartite(H.dual()).plot()

In [12]:
g = hg.hypernetx_to_graphistry_nodes(H)
g._edges.sample(3)

,src,dst,hyperedge
7,BR,BM,__id__4
17,BM,CH,__id__4
5,CC,CH,__id__4


## GitHub Social Network

In [21]:
import pandas as pd

# Download the data at https://www.kaggle.com/femikj/github-social-network
nodes = pd.read_csv("github_target_1.csv")

# Change datatypes
nodes.id = nodes.id.astype(str)
nodes.ml_target = nodes.ml_target.astype(str)

nodes.head(5)

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1


In [22]:
edges = pd.read_csv("github_edges_1.csv")

# Change datatypes
edges = edges.astype(str)

edges.head(5)

,id_1,id_2
0,0,23977
1,1,34526
2,1,2370
3,1,14683
4,1,29982


In [23]:
import plotly.express as px

percent_web, percent_ml = nodes.ml_target.value_counts(normalize=True)
px.histogram(data_frame=nodes, x="ml_target", histnorm="probability density")

In [24]:
edges = edges.merge(nodes, how="left", left_on="id_1", right_on="id").merge(
    nodes, how="left", left_on="id_2", right_on="id", suffixes=("1", "2")
)[["id_1", "id_2", "name1", "name2", "ml_target1", "ml_target2"]]

edges.head(5)

,id_1,id_2,name1,name2,ml_target1,ml_target2
0,0,23977,Eiryyy,airtoxin,0,0
1,1,34526,shawflying,ghosind,0,0
2,1,2370,shawflying,jasondu,0,0
3,1,14683,shawflying,chaoslawful,0,0
4,1,29982,shawflying,dead-horse,0,0


In [25]:
cross_edge = edges.query("ml_target1 != ml_target2")
percentage_cross_edge = cross_edge.shape[0] / edges.shape[0]
percentage_cross_edge

0.1546558340224842

In [26]:
percentage_cross_edge / percent_web

0.20852347708049263

In [27]:
percentage_cross_edge / percent_ml

0.5986779897985065

In [28]:
g = graphistry.edges(edges, "id_1", "id_2")

# Choose icons here: https://fontawesome.com/v4.7.0/icons/
g = (g.nodes(nodes, "id").encode_point_icon(
    "ml_target", categorical_mapping={1: "area-chart", 0: "mouse-pointer"}
))

In [29]:
g.plot()

In [30]:
g2 = g.encode_point_color("ml_target", categorical_mapping={1: "silver", 0: "maroon"})
g2.plot()

#### Detect Communities

In [44]:
%pip install python-louvain


     -------------------------------------- 204.6/204.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9395 sha256=c6302ff25087b4c009495133751799ffc29dd4ce3a4095e42644b37f0bcf2825
  Stored in directory: c:\users\siddh\appdata\local\pip\cache\wheels\af\40\91\ef3635f7d3ddbd889b5af9f1ac22b8d30c48d6f744cd74a6d1
Successfully built python-louvain


In [48]:
from community import community_louvain
import networkx as nx

In [50]:
G = nx.from_pandas_edgelist(edges, "id_1", "id_2")
# compute the best partition
partition = community_louvain.best_partition(G)


In [56]:
parts = pd.DataFrame()
parts['partition'] = list(partition.values())
parts['vertex'] = list(partition.keys())
parts.head(2)

,partition,vertex
0,0,0
1,0,23977


In [66]:
print(sorted(parts.partition.unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]


In [61]:
nodes_part = nodes.merge(parts, how = 'left', left_on = "id", right_on = "vertex")

In [62]:
%pip install faker


     ---------------------------------------- 1.6/1.6 MB 9.1 MB/s eta 0:00:00


In [67]:
from faker import Faker

Faker.seed(0)
fake = Faker()

g3 = (
    g.nodes(nodes_part, "id")
    .encode_point_icon(
        "ml_target", categorical_mapping={1: "area-chart", 0: "mouse-pointer"}
    )
    .encode_point_color(
        "partition", categorical_mapping={i: fake.color() for i in range(59)}
    )
)

g3.plot()

In [68]:
def value_count_per_partition(nodes_part: pd.DataFrame, partition: int):
    """Find the percentage of each developer type per partition"""
    
    part = nodes_part[nodes_part.partition == partition]
    count = part.ml_target.value_counts(normalize=True).values.tolist()
    if len(count) == 2:
        count.append(str(partition))
    else:
        count.extend([0, str(partition)])
    return count

# Get the percentage of each developer type for each partition
count_df = pd.DataFrame(
    [value_count_per_partition(nodes_part, i) for i in range(22)],
    columns=["web_developer", "ml_developer", "partition"],
)

count_df = count_df.melt(
    id_vars=["partition"], var_name="role", value_name="percentage"
)


# Plot 
px.bar(count_df, x="partition", y="percentage", color="role", barmode="group")